In [1]:
import cv2
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

In [2]:
capture = cv2.VideoCapture(r"C:\Users\Jimmy\Documents\Experimental_Forest\Video Data\Location-6\L6V2.mp4")
total_frames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
print(total_frames)
df = pd.read_csv('GT_IMG_1258.csv')
df2 = pd.read_csv('GT_IMG_1258_919.csv')

2218.0


In [3]:
def getROICorners(GT_df, frame_index):
    
    max_width = 375
    max_y = 720 # represents floor in front
    min_y = 230 # represents "horizon"
    slope = max_width / (max_y-min_y)
    point_count = GT_df["#ofPoints"][frame_index]
    frame_points = GT_df.iloc[frame_index, 2:(2+point_count*2)]
    Xlist = []
    Ylist = []
    
    # iterate through each point using the y-value
    for i in range(1, point_count*2, 2):
        # This linear equation serves as a base
        width = max_width - (max_y-frame_points[i]) * slope
        width = int(width//2)
        Xlist.append(frame_points[i-1]+width)
        Ylist.append(frame_points[i])
        Xlist.append(frame_points[i-1]-width)
        Ylist.append(frame_points[i])
    return Xlist, Ylist

In [4]:
def draw_polygon(image, points, color):
    points = [[point[0], point[1]] for point in points]
    points = np.array([points], dtype=np.int32)
    cv2.fillPoly(image, points, color)

In [5]:
def applyMask(frame, polygon, color='green'):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    # Get the coordinates of the polygon
    roi_corners = np.array(polygon.exterior.coords, dtype=np.int32)
    cv2.fillPoly(mask, [roi_corners], (255, 255, 255))

    # Define the mask color based on the input color
    if color == 'green':
        mask_color = (0, 175, 0)
    elif color == 'blue':
        mask_color = (175, 0, 0)
    elif color == 'red':
        mask_color = (0, 0, 175)

    darkened_mask = cv2.bitwise_and(mask, mask, mask=mask)

    # Create an overlay with the same size as the image
    overlay = np.zeros_like(frame)
    overlay[darkened_mask != 0] = mask_color

    masked_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)

    output = cv2.addWeighted(frame, 1, masked_overlay, 0.25, 0.5)
    return output

In [6]:
def calculate_iou(polygon1, polygon2):
    # Clean the polygons to fix any self-intersection issues
    clean_polygon1 = polygon1.buffer(0)
    clean_polygon2 = polygon2.buffer(0)

    # Calculate the intersection and union of the cleaned polygons
    intersection_area = clean_polygon1.intersection(clean_polygon2).area
    union_area = clean_polygon1.union(clean_polygon2).area


    # Check if the union area is zero
    if union_area == 0:
        iou = 0  # Handle the case where there is no union
    else:
        # Calculate the IoU
        iou = intersection_area / union_area
        iou = intersection_area / union_area

    return iou

In [7]:
Xlist1, Ylist1 = getROICorners(df, 0)
points1 = []
for point in range(len(Xlist1)):
    points1.append( (Xlist1[point],Ylist1[point]) )
print(points1)

[(775.0, 700.0), (417.0, 700.0), (641.0, 364.0), (539.0, 364.0), (550.0, 301.0), (496.0, 301.0)]
